In [271]:
import copy
import numpy
import random
import math

In [272]:
GROUP_ID = 'Group04'
ALGORITHM = 'bt'
PUZZLE_TYPE = 'easy'
PUZZLE_PATH = 'puzzles/Easy-P2.txt'

In [273]:
def printPuzzle(puzzle):
    for row in range(9):
        for column in range(8):
            print(puzzle[row][column].value + ",", end = "")
        print(puzzle[row][8].value)
    print("\n")

In [274]:
'''
def generateRandomPuzzle(originalPuzzle):
    ##want to create row consistency at the very least
    newPuzzle = copy.deepcopy(originalPuzzle)
    for row in range(9):
        possibleNums = ['1','2','3','4','5','6','7','8','9']
        #determining the possible values for the row
        for column in range(9):
            tile = newPuzzle[row][column]
            if(tile.fixed):
                possibleNums.remove(tile.value)
        for column in range(9):
            tile = newPuzzle[row][column]
            if not(tile.fixed):
                randomNum = random.choice(possibleNums)
                tile.value = randomNum
                possibleNums.remove(randomNum)
    return newPuzzle
'''
# Fill in the blank spots in the puzzle to create a random puzzle.
# Fill in 3x3 squares at a time such that all values in the square are different
def generateRandomPuzzle(originalPuzzle):
    newPuzzle = copy.deepcopy(originalPuzzle)
    for i in range(3):
        for j in range(3):
            nums = ['1','2','3','4','5','6','7','8','9']
            for row in range(0+(i*3), (i*3)+3):
                for column in range(0+(j*3), (j*3)+3):
                    tile = originalPuzzle[row][column]
                    if (tile.fixed == True):
                        
                        nums.remove(tile.value)
            for row in range(0+(i*3), (i*3)+3):
                for column in range(0+(j*3), (j*3)+3):
                    tile = newPuzzle[row][column]
                    if not(tile.fixed):
                        tile.value = random.choice(nums)
                        nums.remove(tile.value)
    return newPuzzle
    

In [275]:
class sudokuTile:
    def __init__(self, pos, val, layer):
        self.position = pos
        self.layer = layer
        self.value = val
        self.fixed = True
        if val == '?':
            self.fixed = False
            self.domain = ['1','2','3','4','5','6','7','8','9']
        else:
            self.domain = [val]

In [276]:
#adds up all the constraints violated in the puzzle and returns the value
def checkConstraintsViolated(p):
    violations = 0
    # checks if puzzle passes row rule
    for row in range(9):
        nums = []
        for column in range(9):
            nums.append(p[row][column].value)
        newNums = [item for item in nums if item != "?"]
        setNums = set(newNums)
        violations += len(newNums)- len(setNums)

    # checks if puzzle passes column rule
    for column in range(9):
        nums = []
        for row in range(9):
            nums.append(p[row][column].value)
        newNums = [item for item in nums if item != "?"]
        setNums = set(newNums)
        violations += len(newNums)- len(setNums)

    # checks if puzzle passes 3x3 grid rules
    for i in range(3):
        for j in range(3):
            nums = []
            for row in range(0 + (i * 3), (i * 3) + 3):
                for column in range(0 + (j * 3), (j * 3) + 3):
                    nums.append(p[row][column].value)
            newNums = [item for item in nums if item != "?"]
            setNums = set(newNums)
            violations += len(newNums)- len(setNums)
    return violations

In [277]:
def checkPuzzle(p):
    numConstraintsViolated = checkConstraintsViolated(p)
    if(numConstraintsViolated > 0):
        return False
    else:
        return True

In [278]:
#Simple backtracking algorithm using a stack
def backTracking(puzzle):
    #setting up the stack with the inital possible values for the first tile
    stack = [(puzzle[0][0], d) for d in puzzle[0][0].domain]
    layer = 0
    steps = 0
    while len(stack) > 0:
        # pop the top value off of the stack, and set the given tile's value to the given value
        prevLayer = layer
        steps += 1
        action = stack.pop()
        tile = action[0]
        val = action[1]
        tile.value = val
        # if the value is valid for the puzzle, add all the possible values for the next position on the board to the stack
        if (checkPuzzle(puzzle)):
            layer = tile.layer + 1
            # puzzle has been solved
            if (layer == 81):
                break
            row = layer // 9
            col = layer % 9
            nextTile = puzzle[row][col]
            for d1 in nextTile.domain:
                stack.append((nextTile, d1))
        # if the value is not valid, set the tile back to an unknown, and don't continue searching that path (prune said branch)
        else:
            if (not tile.fixed):
                layer = tile.layer
                tile.value = '?'

        # when backtracking, cleanup all unfixed tiles by turning them back into unknowns
        if (prevLayer > layer):
            for i in range(layer + 1, prevLayer):
                row = i // 9
                col = i % 9
                resetTile = puzzle[row][col]
                if not (resetTile.fixed):
                    resetTile.value = '?'

            # fix for edge case where puzzle couldn't backtrack to '1' in the starting square
            if (len(stack) == 0 and puzzle[0][0].value == '?'):
                puzzle[0][0].value = '1'
                nextTile = puzzle[0][1]
                for d1 in nextTile.domain:
                    stack.append((nextTile, d1))
    return puzzle, steps

In [279]:
#Backtracking algorithm with one-step forward checking
def forwardChecking(puzzle):
    #setting up the stack with the inital possible values for the first tile
    stack = [(puzzle[0][0], d) for d in puzzle[0][0].domain]
    layer = 0
    steps = 0
    while len(stack) > 0:
        # pop the top value off of the stack, and set the given tile's value to the given value
        prevLayer = layer
        steps += 1
        action = stack.pop()
        tile = action[0]
        val = action[1]
        tile.value = val
        # if the value is valid for the puzzle, add all the possible values for the next position on the board to the stack
        if (checkPuzzle(puzzle)):
            layer = tile.layer + 1
            # puzzle has been solved
            if (layer == 81):
                break
            row = layer // 9
            col = layer % 9
            nextTile = puzzle[row][col]
            #Changed for forwardTracking-- edits the domain of the next tile if value of current one interferes(same row)
            if(tile.position[0] == nextTile.position[0]):
                checkedDomain = [num for num in nextTile.domain if num != val]
            else:
                checkedDomain = nextTile.domain
            #If there are no numbers in the new domain, the value for the given tile is not possible
            if(len(checkedDomain) == 0):
                layer = tile.layer
                if (not tile.fixed):
                    tile.value = '?'
            for d1 in checkedDomain:
                stack.append((nextTile, d1))
        # if the value is not valid, set the tile back to an unknown, and don't continue searching that path (prune said branch)
        else:
            layer = tile.layer
            if (not tile.fixed):
                tile.value = '?'

        # when backtracking, cleanup all unfixed tiles by turning them back into unknowns
        if (prevLayer > layer):
            for i in range(layer + 1, prevLayer):
                row = i // 9
                col = i % 9
                resetTile = puzzle[row][col]
                if not (resetTile.fixed):
                    resetTile.value = '?'

            # fix for edge case where puzzle couldn't backtrack to '1' in the starting square
            if (len(stack) == 0 and puzzle[0][0].value == '?'):
                puzzle[0][0].value = '1'
                nextTile = puzzle[0][1]
                for d1 in nextTile.domain:
                    stack.append((nextTile, d1))
    return puzzle, steps

In [280]:
# Makes a new puzzle from a given puzzle by swapping two values within one of the 3x3 grids
def getNeighborPuzzle(puzzle):
    neighbor = copy.deepcopy(puzzle)
    startrow = random.choice([0,3,6])
    startcol = random.choice([0,3,6])
    fixed = 0
    for r in range(startrow, startrow + 3):
        for c in range(startcol, startcol + 3):
            if (neighbor[r][c].fixed ==True):
                fixed = fixed + 1
    # Check if 3x3 square has all fixed points
    if fixed >= 8:
        return neighbor
    row1 = random.choice(range(startrow,startrow + 3))
    col1 = random.choice(range(startcol,startcol + 3))
    row2 = random.choice(range(startrow,startrow + 3))
    col2 = random.choice(range(startcol,startcol + 3))

    while(neighbor[row1][col1].fixed == True):
        row1 = random.choice(range(startrow,startrow + 3))
        col1 = random.choice(range(startcol,startcol + 3))
    while(neighbor[row2][col2].fixed == True):
        row2 = random.choice(range(startrow,startrow + 3))
        col2 = random.choice(range(startcol,startcol + 3))

    temp = neighbor[row1][col1]
    neighbor[row1][col1] = neighbor[row2][col2]
    neighbor[row2][col2] = temp
    return neighbor

def simulatedAnnealing(puzzle,maxSteps,temp,a):
    currentPuzzle = generateRandomPuzzle(puzzle)
    k = 1
    cutoff = 1e-4
    for iteration in range(maxSteps):
        t = temp/((iteration+1)*a)
        #print(checkConstraintsViolated(currentPuzzle))
        #printPuzzle(currentPuzzle)
        if (checkConstraintsViolated(currentPuzzle) == 0):
            return currentPuzzle
        if (t < cutoff):
            return currentPuzzle
        nextPuzzle = getNeighborPuzzle(currentPuzzle)
        delta = checkConstraintsViolated(currentPuzzle) - checkConstraintsViolated(nextPuzzle)

        if (delta > 0):
            currentPuzzle = nextPuzzle
        elif (random.random() > math.exp(delta/k*t)):
            currentPuzzle = nextPuzzle

    return currentPuzzle
        
        

In [281]:
def geneticCrossover(firstPuzzle, secondPuzzle, crossoverRate):
    newPuzzle = copy.deepcopy(firstPuzzle)
    for row in range(9):
        for column in range(9):
            randomNum = random.uniform(0, 1)
            if (randomNum <= crossoverRate):
                newPuzzle[row][column].value = secondPuzzle[row][column].value
    return newPuzzle

def geneticMutation(mutationRate, puzzle):
    #want to mutate by picking a random row and swapping two (unfixed) values
    newPuzzle = copy.deepcopy(puzzle)
    randomNum = random.uniform(0, 1)
    if (randomNum <= mutationRate):
        randomRow = random.randrange(0,9)
        foundSwaps = False
        while(not foundSwaps):
            randomCol1 = random.randrange(0,9)
            tile1 = newPuzzle[randomRow][randomCol1]
            randomCol2 = random.randrange(0,9)
            tile2 = newPuzzle[randomRow][randomCol2]
            if not (tile1.fixed or tile2.fixed):
                tempVal = tile1.value
                tile1.value = tile2.value
                tile2.value = tempVal
                foundSwaps = True

    return newPuzzle


def geneticAlgorithm(originalPuzzle, popSize, crossoverRate, mutationRate):
    population = {}
    #initializing the population
    for i in range(popSize):
        #create new puzzle
        randomPuzzle = generateRandomPuzzle(originalPuzzle)
        randomPuzzle = tuple(map(tuple, randomPuzzle))
        #calculate the fitness of the puzzle
        fitness = 1/(checkConstraintsViolated(randomPuzzle))
        population[randomPuzzle] = fitness
    #applying crossover and mutation to individuals to create children
    #number of epochs
    for i in range(100):
        newPopulation = {}
        for puzzle in population:
            selectionPopulation = copy.deepcopy(population)
            #TOURNAMENT:: MAY THE 2 BEST WIN
            tourneySize = 10
            tournament = {}
            for t in range(tourneySize):
                randomChamp = random.choice(list(selectionPopulation.keys()))
                tournament[randomChamp] = selectionPopulation[randomChamp]
                selectionPopulation.pop(randomChamp)
            firstParent = max(tournament.keys(), key=tournament.get)
            tournament.pop(firstParent)
            secondParent = max(tournament.keys(), key=tournament.get)
            newPuzzle = geneticCrossover(firstParent, secondParent, crossoverRate)
            newPuzzle = geneticMutation(mutationRate, newPuzzle)
            #need to calculate loss of new puzzle again and add it to the new population
            fitness = 1/(checkConstraintsViolated(newPuzzle))
            newPuzzle = tuple(map(tuple, newPuzzle))
            newPopulation[newPuzzle] = fitness
        population = newPopulation
        print("Iteration ", i, "least constraints violated:" , 1/max(population.values()))
    finalPuzzle = max(population.keys(), key=population.get)
    return finalPuzzle



In [282]:
#Processes the sudoku file into a numpy array of sudokuTile objects
arr = []
layer = 0
with open(PUZZLE_PATH, 'r') as file:
    firstLine = True
    rowNum = 0
    for line in file:
        colNum = 0
        processedLine = line.rstrip('\n')
        if(firstLine):
            row = processedLine[1:].split(",")
            firstLine = False
        else:
            row = processedLine.split(",")
        tileRow = []
        for num in row:
            tile = sudokuTile([rowNum, colNum], num, layer)
            layer += 1
            tileRow.append(tile)
            colNum += 1
        arr.append(tileRow)
        rowNum += 1
    file.close()
puzzle = numpy.array(arr)
originalPuzzle = copy.deepcopy(puzzle)
if(ALGORITHM == 'bt'):
    solvedPuzzle, steps = backTracking(puzzle)
elif(ALGORITHM == 'fc'):
    solvedPuzzle, steps = forwardChecking(puzzle)
print(ALGORITHM)
print(PUZZLE_PATH)
print("=====================")
print("Original Puzzle: ")
printPuzzle(originalPuzzle)
print("Solved Puzzle in", steps, "steps:")
printPuzzle(solvedPuzzle)


#Writing the finished puzzle to a file
fileName = GROUP_ID + '_' + ALGORITHM + "_" + PUZZLE_TYPE + "_" + PUZZLE_PATH.lstrip("puzzles/")
with open(fileName, 'w') as file:
    for row in range(9):
        for column in range(8):
            file.write(puzzle[row][column].value + ",")
        file.write(puzzle[row][8].value + '\n')
    file.close()

bt
puzzles/Easy-P2.txt
Original Puzzle: 
5,7,?,?,?,?,2,?,?
?,9,?,1,?,?,?,5,7
1,?,?,?,7,?,?,3,6
2,8,?,?,?,6,?,?,4
9,6,?,8,?,5,?,2,3
7,?,?,2,?,?,?,9,8
8,3,?,?,5,?,?,?,9
6,2,?,?,?,7,?,8,?
?,?,7,?,?,?,?,6,2


Solved Puzzle in 6714 steps:
5,7,8,3,6,9,2,4,1
3,9,6,1,2,4,8,5,7
1,4,2,5,7,8,9,3,6
2,8,3,7,9,6,5,1,4
9,6,4,8,1,5,7,2,3
7,1,5,2,4,3,6,9,8
8,3,1,6,5,2,4,7,9
6,2,9,4,3,7,1,8,5
4,5,7,9,8,1,3,6,2




In [283]:
#originalPuzzle = arr
#bestPuzzle = geneticAlgorithm(originalPuzzle, 150, 0.5, .1)
#printPuzzle(bestPuzzle)
#print(checkConstraintsViolated(bestPuzzle))



In [284]:
best = simulatedAnnealing(originalPuzzle,100000,10,.7)
printPuzzle(best)
print(checkConstraintsViolated(best))


5,7,8,9,6,3,2,4,1
3,9,6,1,2,4,8,5,7
1,4,2,5,7,8,9,3,6
2,8,5,7,9,6,6,1,4
9,6,4,8,1,5,7,2,3
7,1,3,2,3,4,5,9,8
8,3,1,6,5,2,4,7,9
6,2,9,1,4,7,3,8,5
4,5,7,3,8,9,1,6,2


4
